# Traveling Salesman Problem

The Traveling Salesman Problem (TSP) is a classic optimization problem that can be tackled using evolutionary algorithms.

**Problem Description**:

- You are given a set of n cities and a distance or cost matrix that provides the distances between each pair of cities.
- The goal is to find the shortest possible tour that visits each city exactly once and returns to the starting city (completing the circuit). Such a tour is called a Hamiltonian cycle.
- The objective is to minimize the total distance or cost of the tour.

**Input**:

- n: The number of cities (n >= 2).
- Distances: A distance matrix D, where D[i][j] represents the distance between city i and city j. The matrix is symmetric, and D[i][i] is usually set to 0 (i.e., the distance from a city to itself is zero).

**Output**:

- A permutation of the cities representing the order in which they should be visited to form the optimal tour.
- The total distance or cost of the optimal tour.

**Constraints**:

- The tour must visit each city exactly once and return to the starting city.
- The problem is typically defined in the Euclidean plane, but it can also be defined in other spaces.
- The goal is to find the shortest possible tour, which is a combinatorial optimization problem.

Solving the TSP using evolutionary algorithms involves representing potential solutions (tours) as permutations of cities and evolving these permutations to find an optimal or near-optimal tour with the shortest distance. Various evolutionary operators like crossover and mutation are applied to generate new tours, and the fitness function calculates the total distance of a tour. The algorithm aims to find a tour with the minimum total distance, which corresponds to the shortest route that visits all cities.

The TSP is a well-known NP-hard problem, and evolutionary algorithms offer a heuristic approach to finding good solutions when an exhaustive search is impractical for large instances.


In [615]:
import random
import numpy as np
import math

In [616]:
# evolution parameters
POPULATION_SIZE = 20
MUTATION_RATE = 0.01
NUMBER_GENERATIONS = 100

# problem parameters
NUMBER_CITIES = 50

In [617]:
def generate_distance_matrix(num_cities, seed=None):
    if seed is not None:
        random.seed(seed)  # Set a seed for reproducibility

    # empty distance matrix filled with zeros
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    # random city coordinates
    city_coordinates = [
        (random.uniform(0, 100), random.uniform(0, 100)) for _ in range(num_cities)
    ]

    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            x1, y1 = city_coordinates[i]
            x2, y2 = city_coordinates[j]
            distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
            distance = int(distance)
            # Populate both symmetric elements of the matrix
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance

    return distance_matrix

In [618]:
# population initialization
def generate_random_tour(num_cities):
    tour = list(range(num_cities))
    random.shuffle(tour)
    return tour

# Fitness


In [619]:
# Fitness function
def fitness(tour, distance_matrix):
    tour_distance = 0
    num_cities = len(tour)

    for i in range(num_cities):
        from_city = tour[i]
        to_city = tour[(i + 1) % num_cities]
        tour_distance += distance_matrix[from_city][to_city]

    return tour_distance

# The New Generation


In [620]:
TOURNAMENT_SIZE = 5


def tournament_selection(population, distance_matrix):
    tournaments_winners = []
    for i in range(len(population)):
        # randomly select k individuals from the population
        tournament = random.sample(population, TOURNAMENT_SIZE)
        # sort the selected individuals according to their fitness
        tournament.sort(key=lambda tour: fitness(tour, distance_matrix))
        # select the best individual from the tournament
        winner = tournament[0]
        tournaments_winners.append(winner)
    return tournaments_winners

In [621]:
CROSSOVER_POINT = NUMBER_CITIES // 2


def reproduce(tournaments_winners):
    new_population = []
    for j in range(POPULATION_SIZE):
        child = random.sample(tournaments_winners, 1)[0]

        if random.random() < MUTATION_RATE:
            # mutate by swapping two cities
            swap_index1, swap_index2 = random.sample(range(NUMBER_CITIES), 2)
            child[swap_index1], child[swap_index2] = (
                child[swap_index2],
                child[swap_index1],
            )

        new_population.append(child)

    return new_population

# Run


In [622]:
distance_matrix = generate_distance_matrix(NUMBER_CITIES, seed=42)

population = [generate_random_tour(NUMBER_CITIES) for i in range(POPULATION_SIZE)]


for i in range(NUMBER_GENERATIONS):
    # select the best parents by tournament selection
    tournaments_winners = tournament_selection(population, distance_matrix)

    # create new population by random mutation
    new_population = reproduce(tournaments_winners)

    population = new_population

# print the best tour
best = sorted(population, key=lambda x: fitness(x, distance_matrix))[0]
print("Best", best)
print("Best tour length", fitness(best, distance_matrix))

Best [10, 35, 34, 21, 11, 9, 31, 22, 7, 3, 38, 6, 16, 48, 20, 36, 30, 0, 2, 44, 32, 18, 12, 5, 24, 8, 4, 41, 49, 33, 23, 25, 46, 45, 1, 39, 27, 17, 40, 19, 14, 29, 43, 42, 26, 28, 15, 47, 13, 37]
Best tour length 2659
